In [17]:
import os
os.chdir('../..')
from pipelines.util import *
import pandas as pd
from datetime import datetime

In [18]:
data = pd.read_csv(os.path.join(WDIR, 'true-north/true_north_may_2024_clean.csv'))

In [19]:
data.describe()

,last_updated,Create Date,Do you feel the True North report identified the key challenges and opportunities facing the region?,Are you interested in attending future True North events?,Are you currently a B Corp or in the process of becoming a B Corp?,Would you be interested in hearing more from Brabners about the B Corp process?,Company name,City,Industry,sector,location,Which theme of the True North report do you most identify with and could support activity around?,How would you like to be involved with the True North network?,company_size
count,188,188,188,187,186,186,176,141,132,188,47,188,47,47
unique,177,161,2,2,2,2,165,70,50,146,19,13,7,7
top,2022-03-28 00:29:00,2022-03-28 00:29:00,Yes,Yes,No,No,Brabners LLP,London,Legal Services,Legal,Lancashire,"People, skills and the future",Keeping informed about the latest True North n...,0-9
freq,7,21,181,185,157,100,5,19,12,7,9,58,18,21


In [20]:
company_names = pd.Series(data['Company name'].sort_values())
print(company_names.unique())

['2030hub' 'AHR' 'Abeceder' 'Access Creative College'
 'Accountable Recruitment' 'Advanced Manufacturing Research Centre'
 'Agent Marketing Ltd' 'Agile Automations' 'Armstrong Watson LLP' 'Azets'
 'B-Engineering Group' 'BHP LLP'
 'Bank of England (Agency for the North West)' 'Barton Legal Limited'
 'Bicical' 'Big Hand 4 Business' 'Blackpool Council' 'Brabners LLP'
 'Bruntwood' 'Business Doctors' 'Business Health Institute'
 'Business of Science' 'CBI' 'Carbon Happy World' 'Cazenove Capital'
 'Chris Shakespeare Coaching' 'Codurance' 'Cortus Advisory Group Limited'
 'Coverdale Barclay' 'DES Group UK' 'Daneel Williams' 'Deloitte LLP'
 'E-Rail' 'ESGmark' 'Elevategm' 'Endless LLP' 'Essential-Sourcing' 'F-LEX'
 'Farrans (Construction) Limited' 'Finance Yorkshire' 'Fleetsolve Ltd'
 'Fort Search' 'Fox Brothers' 'Fragomen' 'Freshfield'
 'Giles Metcalfe Digital' 'Go-International' 'Gripple Limited'
 'Group Rhodes' 'Groupsignature' 'HSBC' 'Handelsbanken'
 'Harrowells Limited' 'Hawthorne & Burman'

There is LLoyds banking group and lloyds bank plc - we're assuming these are different companies.

In [21]:
data['month'] = pd.to_datetime(data['Create Date'])
data['month_formatted'] = data['month'].apply(datetime.strftime, format='%Y-%m').sort_index(ascending=True)
monthly_members = pd.DataFrame(data['month_formatted'].value_counts(ascending=False)).reset_index()
data.sort_values(by='Create Date').head()

,last_updated,Create Date,Do you feel the True North report identified the key challenges and opportunities facing the region?,Are you interested in attending future True North events?,Are you currently a B Corp or in the process of becoming a B Corp?,Would you be interested in hearing more from Brabners about the B Corp process?,Company name,City,Industry,sector,location,Which theme of the True North report do you most identify with and could support activity around?,How would you like to be involved with the True North network?,company_size,month,month_formatted
11,2024-04-26 16:12:00,2022-01-28 11:00:00,Yes,Yes,Yes,Yes,Brabners LLP,Liverpool,Legal Services,"Legal, Tech, IP",NaN,Innovation and change,NaN,NaN,2022-01-28 11:00:00,2022-01
24,2024-04-09 14:00:00,2022-01-28 13:01:00,Yes,Yes,Yes,Yes,Brabners LLP,Liverpool,Legal Services,Real estate,NaN,"People, skills and the future",NaN,NaN,2022-01-28 13:01:00,2022-01
58,2024-01-23 14:11:00,2022-01-28 15:01:00,Yes,Yes,Yes,No,Brabners LLP,Liverpool,Legal Services,Legal,Cumbria; West Yorkshire; Greater Manchester; M...,Innovation and change,Keeping informed about the latest True North n...,500-999,2022-01-28 15:01:00,2022-01
88,2023-11-01 10:03:00,2022-01-31 20:01:00,Yes,Yes,Yes,No,Brabners LLP,Liverpool,Legal Services,Agriculture and real estate,NaN,Sustainable development,NaN,NaN,2022-01-31 20:01:00,2022-01
2,2024-05-13 11:12:00,2022-03-15 11:01:00,Yes,Yes,Yes,Yes,Brabners LLP,Liverpool,Legal Services,Construction and Engineering,NaN,"People, skills and the future",NaN,NaN,2022-03-15 11:01:00,2022-03


In [22]:
# monthly_members['decimal_date'] = monthly_members['month'].apply(lambda x: x[5:7])
# monthly_members['decimal_date']

In [23]:
monthly_members['timestamp'] = pd.to_datetime(monthly_members['month_formatted'], format='%Y-%m').astype(int) / 10**9

monthly_members['year'] = monthly_members['timestamp'].div((86400*365.25)).add(1970).round(2)

In [24]:
monthly_members.drop(columns='timestamp', inplace=True)
monthly_members.set_index(['year', 'month_formatted'], inplace=True, append=True)
cumsum = pd.DataFrame(monthly_members.sort_index(level=2).cumsum().reset_index().drop(columns='level_0'))
cumsum.to_csv(os.path.join(SRC_DIR,'themes/true-north/membership/_data/cumsum_members.csv'), index=False)